In [40]:
from datasets import load_dataset

In [41]:
dataset = load_dataset("glue", "sst2")
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

In [42]:
for item in train_dataset:
    if item['label'] == 0:
        print(item['sentence'])
        print()

hide new secretions from the parental units 

contains no wit , only labored gags 

remains utterly satisfied to remain the same throughout 

on the worst revenge-of-the-nerds clichés the filmmakers could dredge up 

that 's far too tragic to merit such superficial treatment 

a depressed fifteen-year-old 's suicidal poetry 

goes to absurd lengths 

for those moviegoers who complain that ` they do n't make movies like they used to anymore 

the part where nothing 's happening , 

saw how bad this movie was 

lend some dignity to a dumb story 

cold movie 

redundant concept 

excruciatingly unfunny and pitifully unromantic 

which half of dragonfly is worse : the part where nothing 's happening , or the part where something 's happening 

the plot is nothing but boilerplate clichés from start to finish , 

the action is stilted 

will find little of interest in this film , which is often preachy and poorly acted 

by far the worst movie of the year 

sit through , 

oblivious to the e


analyze this , not 

sounds like horrible poetry 

kurys seems intimidated by both her subject matter and the period trappings of this debut venture into the heritage business . 

most charmless 

befuddled in its characterizations as it begins to seem as long as the two year affair which is its subject 

a dull girl , 

we only know as an evil , monstrous lunatic 

the cumulative effect of the movie is repulsive and depressing . 

make fun of me for liking showgirls 

mediocre 

to see this turd squashed under a truck , preferably a semi 

bore that tends to hammer home every one of its points 

no solace here , no entertainment value 

is confused in death to smoochy into something both ugly and mindless . 

get lost in the murk of its own making 

woozy 

is its reliance on formula , though 

weighted down with slow , uninvolving storytelling and flat acting . 

why did they deem it necessary to document all this emotional misery ? 

our culture is headed down the toilet with the f

, it is n't much fun . 

feeling like a great missed opportunity 

of hard-bitten , cynical journalists 

skip the film and buy the philip glass soundtrack cd . 

every conceivable mistake a director could make in filming opera 

is why anthony hopkins is in it . 

worn-out 

pouty-lipped poof 

difficult and 

its nauseating spinning credits sequence 

is paced at a speed that is slow to those of us in middle age and deathly slow to any teen . 

somebody might devote time to see it 

murphy do the genial-rogue shtick to death 

fatal for a film that relies on personal relationships 

all-over-the-map movie would be a lot better if it pared down its plots and characters to a few rather than dozens ... or if it were subtler ... or if it had a sense of humor . 

if `` gory mayhem '' is your idea of a good time 

, it 's the worst movie i 've seen this summer 

realizing that you 've spent the past 20 minutes looking at your watch 

its ultimate demise 

its hackneyed and meanspirited sto

sordid of human behavior on the screen , 

are very , very good reasons for certain movies to be sealed in a jar and left on a remote shelf indefinitely . 

the uncanny , inevitable and seemingly shrewd facade of movie-biz farce 

the plot holes 

wending its way to an uninspired philosophical epiphany 

the kind of parent who enjoys intentionally introducing your kids to films which will cause loads of irreparable damage that years and years of costly analysis could never fix 

the story plays out slowly 

cliched and 

unpleasant debate that 's been given the drive of a narrative and that 's been acted out 

in a way that bespeaks an expiration date passed a long time ago 

angry and sad 

if the screenplay falls somewhat short 

mattei 's underdeveloped effort 

padded . 

could keep a family of five blind , crippled , amish people alive in this situation better than these british soldiers do at keeping themselves kicking 

loses its ability to shock and amaze . 

b.s. 

is little e


... it feels like a glossy rehash . 

been allowed to use the word `` new '' in its title , because there 's not an original character , siuation or joke in the entire movie 

just seem like a bad idea from frame one 

be hard pressed to think of a film more cloyingly sappy than evelyn this year 

while the film is competent , it 's also uninspired , lacking the real talent and wit to elevate it beyond its formula to the level of classic romantic comedy to which it aspires . 

tell almost immediately that welcome to collinwood is n't going to jell 

dull and ankle-deep 

aimless , arduous 

but only among those who are drying out from spring break 

insignificance 

as it is interminable 

it 's like going to a house party and watching the host defend himself against a frothing ex-girlfriend . 

to some eyes this will seem like a recycling of clichés , an assassin 's greatest hits . 

unlikable , uninteresting , unfunny , and completely 

rehashes 

is a little scattered -- ditsy , 



...

has the thrown-together feel of a summer-camp talent show : hastily written , underrehearsed , arbitrarily plotted 

its audacious ambitions sabotaged by pomposity , steven soderbergh 's space opera emerges as a numbingly dull experience . 

does n't work 

the unsalvageability 

kung pow seems like some futile concoction that was developed hastily after oedekerk and his fellow moviemakers got through crashing a college keg party 

that are damned in queen of the damned 

it 's loud and boring ; watching it is like being trapped at a bad rock concert 

will need to adhere more closely to the laws of laughter 

straight out of a cellular phone commercial 

harm 

so many false scares 

regret and 

struck me as unusually and unimpressively fussy and pretentious 

murk 

confusing on one level or another , 

nightmare 

with a contemptible imitator starring a `` snl '' has-been acting like an 8-year-old channeling roberto benigni 

loneliness can make people act weird 

a loud , witless 

leaves no southern stereotype unturned 

kill michael myers for good : stop buying tickets to these movies 

consigned to the dustbin of history 

perry 's good and his is an interesting character , but `` serving sara '' has n't much more to serve than silly fluff 

, as you watch them clumsily mugging their way through snow dogs , seems inconceivable 

howard and his co-stars all give committed performances , but they 're often undone by howard 's self-conscious attempts to find a ` literary ' filmmaking style to match his subject . 

an idea buried somewhere inside its fabric , but never clearly seen or felt 

hopeless , unsatisfying muddle 

for their acting chops , but for their looks 

visual and verbal clichés 

gore 

bang your head on the seat in front of you , at its cluelessness , at its idiocy 

nothing too deep or substantial . 

invented to describe exactly how bad it is 

to appearing in this junk that 's tv sitcom material at best 

near virtuosity in its crapulence 

s

feels aimless for much of its running time , until late in the film when a tidal wave of plot arrives , leaving questions in its wake . 

a florid biopic about mad queens , obsessive relationships , and rampant adultery 

one-dimensional characters 

drama , 

the film feels homogenized and a bit contrived , as if we 're looking back at a tattered and ugly past with rose-tinted glasses . 

these brats will ever be anything more than losers 

above easy , cynical potshots at morally bankrupt characters 

this is the opposite of a truly magical movie . 

to pack raw dough in my ears 

a maker of softheaded metaphysical claptrap 

flashy , overlong soap opera . 

does n't manage to hit all of its marks 

two whole hours 

like a cold old man going through the motions 

complete with loads of cgi and bushels of violence , but not a drop 

drown a viewer in boredom 

try paying less attention to the miniseries and more attention to the film it is about 

an ungainly movie , ill-fitting , 



not only does the thoroughly formulaic film represent totally exemplify middle-of-the-road mainstream 

of relentlessly nasty situations 

it makes me feel weird / thinking about all the bad things in the world / like puppies with broken legs / and butterflies that die / and movies starring pop queens 

drag audience enthusiasm 

obscenely 

a terrible story 

a dime in the tries-so-hard-to-be-cool `` clockstoppers 

rotoscope animation for no apparent reason except 

airless , prepackaged julia roberts wannabe 

be profane 

a distinctly minor effort that will be seen to better advantage on cable , especially considering its barely feature-length running time of one hour . 

has not a trace of humanity or empathy 

it was n't the subject matter that ultimately defeated the film ... it was the unfulfilling , incongruous , `` wait a second , did i miss something ? '' 

far less enjoyable 

any flaws that come later 

this date-night diversion 

is one reason it 's so lackluster . 

can 

to do , too little time to do it in 

with revulsion 

the film gets added disdain for the fact that it is nearly impossible to look at or understand . 

anti-feminist 

dull . 

it is more likely to induce sleep than fright . 

may not 

imitations 

`` ... something appears to have been lost in the translation this time 

the slack execution italicizes the absurdity of the premise 

it does n't make for great cinema , 

scoob and shag do n't eat enough during the film . ' 

final whistle 

do n't think this movie loves women at all 

through the horrible pains of a death 

a journey that is as difficult for the audience to take as it is for the protagonist 

even the hastily and amateurishly drawn animation can not engage . 

problem is , we have no idea what in creation is going on . 

so exaggerated and broad that it comes off as annoying rather than charming . 

it 's an odd show , pregnant with moods , stillborn except as a harsh conceptual exercise . 

illogic 

the pool with an

### Use BERT to have single vector for sentence

In [43]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [10]:
# Load pre-trained BERT model and tokenizer
# model_name = "bert-base-uncased"
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertModel.from_pretrained(model_name)

In [44]:
# def get_doc_embedding(document, tokenizer, model):
#     # Tokenize input text
#     inputs = tokenizer(document, return_tensors="pt", padding=True, truncation=True)
#     # Forward pass through the model
#     with torch.no_grad():
#         outputs = model(**inputs)
#     # Extract document embedding from hf model output
#     document_embedding = outputs.last_hidden_state.mean(dim=1).squeeze(0)
#     return document_embedding.reshape(-1, 1)

def get_doc_embedding(document, model):
    return model.encode(document)

In [45]:
# Example document from GLUE SST dataset
document = "those so-so films that could have been much better"
doc_emb = get_doc_embedding(document, model)
print(doc_emb.shape)


(768,)


### Save the vectors in pkl files

### Use PCA to reduce vector dimensions

In [61]:
import tqdm
import numpy as np
import pickle

In [62]:
doc_vectors = [] 
# Init dummylist to decode whihc split train/val/test
dummy_list = train_dataset

In [64]:
for item in tqdm.tqdm(dummy_list):
    sentence = item['sentence']
    doc_emb = get_doc_embedding(document, model)
    doc_vectors.append(doc_emb)

100%|██████████| 67349/67349 [41:01<00:00, 27.36it/s]  


In [67]:
doc_vectors[0].shape

(768,)

In [68]:
# Open the file and store the reduced sentence vectors
with open('sst-train-768.pkl', 'wb') as file:
    # Serialize and write the variable to the file
    pickle.dump(doc_vectors, file)

### Extra

In [37]:
# doc_vectors_np = np.array(doc_vectors)

# # 768 is emb dimension
# doc_vec_rehsaped = doc_vectors_np.reshape((len(dummy_list),768))

# # Apply PCA for dimensionality reduction
# pca = PCA(n_components = target_dimension)
# pca.fit(doc_vec_rehsaped)

# final_vecs = pca.transform(doc_vec_rehsaped)